# 手动实现线性回归

In [11]:
%matplotlib inline
import torch
import numpy as np
import matplotlib.pyplot as plt

In [148]:
# 假设w = [2.3, 4.7], b=3.1
# 生成训练数据
sample_nums = 1000
input_nums = 2

X = torch.randn(sample_nums, input_nums)
true_w = torch.tensor([3.2, 4.7], dtype=torch.float32)
true_b = 3.1
y = torch.matmul(X, w.T) + b + torch.tensor(np.random.normal(0, 0.1, size=sample_nums), dtype=torch.float32)

In [149]:
# 可视化训练数据
# plt.plot(X[:, 1].numpy(), y.numpy(), '*') # w[1]/w[0]越大，线性关系越明显

In [150]:
class DataGenerator():
    def __init__(self, dataset, label, batch_size):
        self.batch_header = 0
        self.dataset = dataset
        self.label = label
        self.data_len = len(self.dataset)
        self.batch_size = batch_size

    def get_next_batch(self):
        if self.batch_header + self.batch_size < self.data_len:
            self.batch_header += self.batch_size
            return self.dataset[self.batch_header:self.batch_header+self.batch_size, :], self.label[self.batch_header:self.batch_header+self.batch_size]
        else:
            self.batch_header = self.data_len
            return self.dataset[self.batch_header:, :], self.label[self.batch_header:]
            
    def reset(self):
        self.batch_header = 0

In [151]:
# 定义损失计算函数
def loss(true_label, pred_label):
    return (true_label - pred_label.view(true_label.size())) ** 2 / 2

In [152]:
# 定义优化算法
def sgd(params, lr, batch_size):
    for param in params:
        param.data -= param.grad * lr / batch_size

In [153]:
# 定义net
def linreg(X, w, b):
    return torch.mm(X, w) + b

In [154]:
def train(data_generator, epochs, w, b, batch_size, lr, X, y):
    net = linreg 
    for epoch in range(epochs):
        while True:
            train_X, train_y = data_generator.get_next_batch()
            if len(train_X) == 0:
                break
            temp = net(train_X, w.T, b)
            losses = loss(train_y, temp).sum()
            
            losses.backward()

            sgd([w, b], lr, batch_size)


            cur_w.grad.data.zero_()
            cur_b.grad.data.zero_()
        train_loss = loss(y, net(X, w.T, b))
        print('epoch %d, loss %f' % (epoch, train_loss.mean().item()))
        data_generator.reset()

In [155]:
# 初始化参数
cur_w = torch.zeros((1, 2))
cur_b = torch.ones(1)
num_epochs = 10
batchsize = 32
data_generator = DataGenerator(X, y, batchsize)
lr = 0.01
cur_w.requires_grad = True
cur_b.requires_grad = True
train(data_generator, num_epochs, cur_w, cur_b, batchsize, lr, X, y)

epoch 0, loss 10.226126
epoch 1, loss 5.332829
epoch 2, loss 2.785255
epoch 3, loss 1.457646
epoch 4, loss 0.765068
epoch 5, loss 0.403349
epoch 6, loss 0.214194
epoch 7, loss 0.115142
epoch 8, loss 0.063196
epoch 9, loss 0.035910


In [156]:
print('parameter w[0] is %f' % cur_w[0][0].item())
print('parameter w[1] is %f' % cur_w[0][1].item())
print('parameter b is %f' % cur_b.item())

parameter w[0] is 3.111917
parameter w[1] is 4.502316
parameter b is 2.991582


# 利用Pytorch进行实现

In [120]:
import torch
import numpy as np
import torch.utils.data as Data
from torch import nn
import torch.optim as optim

In [139]:
# 生成训练数据
true_w = torch.tensor([2.1, -3.2], dtype=torch.float32)
true_b = torch.tensor(3.2, dtype=torch.float32)

feature_num = 2
sample_num = 1000

X = torch.randn(sample_num, feature_num, dtype=torch.float32)
y = torch.matmul(X, true_w) + true_b + torch.tensor(np.random.normal(0, 0.01, sample_num), dtype=torch.float32)

In [140]:
class Net(nn.Module):
    def __init__(self, feature_num):
        super(Net, self).__init__()
        self.linear = nn.Linear(feature_num, 1)

    def forward(self, x):
        return self.linear(x)

In [141]:
net = Net(feature_num)
nn.init.normal_(net.linear.weight, mean=0, std=0.1)
nn.init.constant_(net.linear.bias, val=0)

loss = nn.MSELoss()

optimizer = optim.SGD(net.parameters(), lr=0.03)

batch_size = 32
dataset = Data.TensorDataset(X, y)
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

In [142]:
epochs = 10

for epoch in range(epochs):
    for X, y in data_iter:
        output = net(X)
        l = loss(y.view(-1, 1), output)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()

    print('epoch %d loss %f' % (epoch, l.item()))

epoch 0 loss 0.510093
epoch 1 loss 0.012354
epoch 2 loss 0.000209
epoch 3 loss 0.000107
epoch 4 loss 0.000086
epoch 5 loss 0.000037
epoch 6 loss 0.000086
epoch 7 loss 0.000191
epoch 8 loss 0.000060
epoch 9 loss 0.000108
